Q1.5 
This query aims to expand the FA-ensemblage gene set based on upstream TF binding site motif patterns.

 - FA gene set  'FA_NCBIGene_symbol.txt'
 - motif simalarity datastore 'jaspar.nt'
 - 
 

In [1]:
import csv
import yaml
import requests
import json
import re

untill the hackathon blazegraph instance is available    
I am running a local instance [described here](http://localhost:8888/notebooks/LocalBlazeGraph.ipynb)





In [2]:
bg_host = 'http://localhost:9999'
bg = bg_host + '/blazegraph/sparql?format=json&query=' 

which offers:  

    rdf:Description rdf:nodeID="service"
        rdf:type 
            rdf:resource="http://www.w3.org/ns/sparql-service-description#Service"
        endpoint 
            rdf:resource="http://localhost:9999/blazegraph/sparql"
        endpoint 
            rdf:resource="http://localhost:9999/bigdata/LBS/sparql"
            
Trying the blazegraph/sparql endpoint yields

In [3]:
x = requests.get(bg + 'SELECT ?g1 WHERE{?g1 <http://purl.obolibrary.org/obo/SO_adjacent_to> ?o} LIMIT 1')
print(x.text)

{
  "head" : {
    "vars" : [ "g1" ]
  },
  "results" : {
    "bindings" : [ {
      "g1" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/27127"
      }
    } ]
  }
}


Other queries which work in the blazegraph web interface at 
http://localhost:9999/blazegraph/#query
include
```
PREFIX SO: <http://purl.obolibrary.org/obo/SO_>
PREFIX RO: <http://purl.obolibrary.org/obo/RO_>
PREFIX SWO: <http://www.ebi.ac.uk/efo/swo/SWO_>
PREFIX NCBIGene: <http://www.ncbi.nlm.nih.gov/gene/>

SELECT (COUNT(?gene) as ?count)
  WHERE {
  	?gene SO:adjacent_to ?region1 .
    ?region1 RO:0002350 ?pairwise .
    ?region2 RO:0002350 ?pairwise .
    FILTER(?region1 != ?region2)
    ?gene2 SO:adjacent_to ?region2 .
    FILTER(?gene != ?gene2)
	?pairwise SWO:0000425 '1' .
  }

count
19790
```
SPARQL path notation works

```    
    select ?g1 ?g2
	    WHERE {
        ?g1 SO:adjacent_to/RO:0002350/^RO:0002350/^SO:adjacent_to ?g2 .
        FILTER(?g1 != ?g2)
      }
    LIMIT 10
```
````
g1	                                        g2
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/3646>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/23077>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/55269>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/168850>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/5635>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/11344>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/29104>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/9578>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/4201>
<http://www.ncbi.nlm.nih.gov/gene/64854>	<http://www.ncbi.nlm.nih.gov/gene/7024>
```

```
look for genes related to FANCA

select ?g2
	WHERE {
  	NCBIGene:2175 SO:adjacent_to/RO:0002350/^RO:0002350/^SO:adjacent_to ?g2 .
    FILTER(NCBIGene:2175 != ?g2)
  }
```


```
g2
<http://www.ncbi.nlm.nih.gov/gene/8795>
<http://www.ncbi.nlm.nih.gov/gene/348995>
<http://www.ncbi.nlm.nih.gov/gene/5166>
<http://www.ncbi.nlm.nih.gov/gene/23155>
<http://www.ncbi.nlm.nih.gov/gene/7568>
<http://www.ncbi.nlm.nih.gov/gene/441381>
<http://www.ncbi.nlm.nih.gov/gene/55147>
<http://www.ncbi.nlm.nih.gov/gene/8242>
<http://www.ncbi.nlm.nih.gov/gene/54830>
<http://www.ncbi.nlm.nih.gov/gene/22976>
<http://www.ncbi.nlm.nih.gov/gene/23193>
<http://www.ncbi.nlm.nih.gov/gene/57623>
<http://www.ncbi.nlm.nih.gov/gene/6093>
<http://www.ncbi.nlm.nih.gov/gene/54929>
<http://www.ncbi.nlm.nih.gov/gene/57577>
<http://www.ncbi.nlm.nih.gov/gene/9131>
<http://www.ncbi.nlm.nih.gov/gene/54955>
<http://www.ncbi.nlm.nih.gov/gene/51645>
<http://www.ncbi.nlm.nih.gov/gene/8659>
<http://www.ncbi.nlm.nih.gov/gene/51645>
<http://www.ncbi.nlm.nih.gov/gene/6093>
<http://www.ncbi.nlm.nih.gov/gene/81551>
<http://www.ncbi.nlm.nih.gov/gene/57623>
<http://www.ncbi.nlm.nih.gov/gene/29028>
<http://www.ncbi.nlm.nih.gov/gene/90957>
<http://www.ncbi.nlm.nih.gov/gene/9517>
<http://www.ncbi.nlm.nih.gov/gene/3376>
<http://www.ncbi.nlm.nih.gov/gene/606495>
<http://www.ncbi.nlm.nih.gov/gene/606495>
<http://www.ncbi.nlm.nih.gov/gene/90957>
<http://www.ncbi.nlm.nih.gov/gene/55147>
<http://www.ncbi.nlm.nih.gov/gene/57577>
<http://www.ncbi.nlm.nih.gov/gene/7568>
<http://www.ncbi.nlm.nih.gov/gene/3376>
<http://www.ncbi.nlm.nih.gov/gene/9517>
<http://www.ncbi.nlm.nih.gov/gene/54756>
<http://www.ncbi.nlm.nih.gov/gene/23193>
<http://www.ncbi.nlm.nih.gov/gene/348995>
<http://www.ncbi.nlm.nih.gov/gene/23155>
<http://www.ncbi.nlm.nih.gov/gene/6604>
<http://www.ncbi.nlm.nih.gov/gene/81551>
<http://www.ncbi.nlm.nih.gov/gene/54785>
<http://www.ncbi.nlm.nih.gov/gene/55171>
<http://www.ncbi.nlm.nih.gov/gene/9928>
<http://www.ncbi.nlm.nih.gov/gene/6566>
<http://www.ncbi.nlm.nih.gov/gene/22976>
<http://www.ncbi.nlm.nih.gov/gene/5232>
<http://www.ncbi.nlm.nih.gov/gene/84650>
<http://www.ncbi.nlm.nih.gov/gene/5166>
<http://www.ncbi.nlm.nih.gov/gene/255426>
```



Next, move that behavior into this jupyter notebook

In [4]:
# Want to use existing translation tables when constructing SPARQL queries
yamlurl='https://raw.githubusercontent.com/TomConlin/Jaspar_FA/master/translation_tables/curie_map.yaml'
rsponse = requests.get(yamlurl)
PREFIX = yaml.load(rsponse.text)
# print(PREFIX)

yamlurl='https://raw.githubusercontent.com/TomConlin/Jaspar_FA/master/translation_tables/translation_table.yaml'
rsponse = requests.get(yamlurl)
TT = yaml.load(rsponse.text)
# print(TT)

# redecorate the curie base IRI map 
# to look like a sparql prefix namespace map  
# except the bnode: skolemIRI which is given a java 
#     "org.openrdf.query.MalformedQueryException:
# for no good reason

prefixns = ""
for p in PREFIX: 
    if len(p) > 1:
       prefixns += 'PREFIX ' + p + ': <' + PREFIX[p] + '>\n'

print(prefixns)

PREFIX OIO: <http://www.geneontology.org/formats/oboInOwl#>
PREFIX RO: <http://purl.obolibrary.org/obo/RO_>
PREFIX JASPAR: <http://fantom.gsc.riken.jp/5/sstar/JASPAR_motif:>
PREFIX SO: <http://purl.obolibrary.org/obo/SO_>
PREFIX GENO: <http://purl.obolibrary.org/obo/GENO_>
PREFIX SWO: <http://www.ebi.ac.uk/efo/swo/SWO_>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX NCBIGene: <http://www.ncbi.nlm.nih.gov/gene/>



The Fanconi Anemia genes come as symbols/aliases in three sets [here](https://docs.google.com/spreadsheets/d/1yX-5sfrC3vrahf4_k7-5rl4Oqzm853ollIMmUo1PTc0)  
I converted them to NCBI gene identifiers and current symbols [here](https://github.com/NCATS-Tangerine/cq-notebooks/tree/master/FA_gene_sets)

In [5]:

fagene=[]

fa1 = '../FA_gene_sets/FA_1_core_complex.txt'
core_complex = {}
with open(fa1, 'r') as tabfile:
    filereader = csv.reader(tabfile, delimiter='\t')
    for row in filereader:   
        (fa_gene, fa_symbol) = row
        fagene.append(fa_gene)
        core_complex[fa_gene]=fa_symbol
        
fa2 = '../FA_gene_sets/FA_2_effector_proteins.txt'
effector_proteins = {}
with open(fa2, 'r') as tabfile:
    filereader = csv.reader(tabfile, delimiter='\t')
    for row in filereader:   
        (fa_gene, fa_symbol) = row
        fagene.append(fa_gene)
        effector_proteins[fa_gene]=fa_symbol
        
fa3 = '../FA_gene_sets/FA_3_associated_proteins.txt'
associated_proteins = {} 
with open(fa3, 'r') as tabfile:
    filereader = csv.reader(tabfile, delimiter='\t')
    for row in filereader:   
        (fa_gene, fa_symbol) = row
        fagene.append(fa_gene)
        associated_proteins[fa_gene]=fa_symbol

We can get all the triples patterns available in 'jaspar.nt'   
from this [GraphViz dot file](https://github.com/TomConlin/Jaspar_FA/blob/master/jaspar_target_model.gv) since we used it to guide   
generating the RDF data in the first place.  

![](https://github.com/TomConlin/Jaspar_FA/blob/master/jaspar_target_model.png?raw=true)



Composing the query by atomizing the GraphViz elements   
and selectivly translating with the same tables   
the data was generates with means  
the SPARQL query remains free of semanticly obsolete clauses.

I did this manually, but in general, the rules were roughly:
  
-    remove angle brackets
-    add trailing dot 
-    replace BNODE: with __?__
-    if the element (predicate) is a curie then wrap it in a TT lookup
-    if the object is a LITERAL wrap it in quotes
-    give the items of intrest specific __?names__
-    formating to suit sensibilities

In particular we want: 
    candidate genes which share motif similarity with FA genes
    
    <NCBIGene:fagene><SO:adjacent_to><BNODE:gene1_upstream_region>
    <BNODE:gene1_upstream_region><RO:member of><BNODE:pairwise_similarity>
    <BNODE:gene2_upstream_region><RO:member of><BNODE:pairwise_similarity>
    <NCBIGene:xyz><SO:adjacent_to><BNODE:gene2_upstream_region> 
    
    when <NCBIGene:xyz> is not <NCBIGene:fagene> 
    
we may also be interested in limiting by region extent size   
or Jaccard similarity score

    <BNODE:pairwise_similarity><SWO:Similarity score><0.73>
    <BNODE:gene1_upstream_region><GENO:has_extent><1k>
    
    

Much of the effort to this point has been developing and maintaining a context  
in which we are able to __write a readable query__.  

Here, given a (fa) gene, we are looking for other genes   
with optimal matches (similarity=1) within their 1k start regions.

In [27]:
selectstr = ' '.join([
    'SELECT ?fagene ?candidate\n',
    'WHERE{\n',
        '?fagene', 'SO:adjacent_to', '?gene1_upstream_region .\n', 
        '?gene1_upstream_region',  TT['RO:member of'], '?pairwise_similarity .\n',
        '?gene2_upstream_region',  TT['RO:member of'], '?pairwise_similarity .\n',
        'FILTER(?gene1_upstream_region != ?gene2_upstream_region) \n', 
        '?candidate', 'SO:adjacent_to', '?gene2_upstream_region .\n',
        '?pairwise_similarity', TT['SWO:Similarity score'], "'1' .\n", 
        '?gene1_upstream_region', TT['GENO:has_extent'], "'1k' .\n",
    'FILTER(?fagene != ?candidate)\n}'
    ]) 

# note the abscence of opaque identifiers the query engine actually uses

query = prefixns + "\n" + selectstr
print(query)

PREFIX OIO: <http://www.geneontology.org/formats/oboInOwl#>
PREFIX RO: <http://purl.obolibrary.org/obo/RO_>
PREFIX JASPAR: <http://fantom.gsc.riken.jp/5/sstar/JASPAR_motif:>
PREFIX SO: <http://purl.obolibrary.org/obo/SO_>
PREFIX GENO: <http://purl.obolibrary.org/obo/GENO_>
PREFIX SWO: <http://www.ebi.ac.uk/efo/swo/SWO_>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX NCBIGene: <http://www.ncbi.nlm.nih.gov/gene/>

SELECT ?fagene ?candidate
 WHERE{
 ?fagene SO:adjacent_to ?gene1_upstream_region .
 ?gene1_upstream_region RO:0002350 ?pairwise_similarity .
 ?gene2_upstream_region RO:0002350 ?pairwise_similarity .
 FILTER(?gene1_upstream_region != ?gene2_upstream_region) 
 ?candidate SO:adjacent_to ?gene2_upstream_region .
 ?pairwise_similarity SWO:0000425 '1' .
 ?gene1_upstream_region GENO:0000678 '1k' .
 FILTER(?fagene != ?candidate)
}


In [36]:
# apply our query to each gene in the FA Core complex
candidate_set = {}
for fagene in core_complex:
    payload = {
        'format' : 'json', 
        # '$fagene':  fagene, 
        # BG is not accepting the curie. Wants IRI or LITERAL
        '$fagene': '<' + re.sub('NCBIGene:',PREFIX['NCBIGene'],fagene) + '>',
        'query': query
    }
    response = requests.post(bg_host + '/blazegraph/sparql', data=payload)
 
    resp = json.loads(response.text)
    if resp['results']['bindings'] != []:
        print(core_complex[fagene], '\t', 
              re.sub('NCBIGene:', PREFIX['NCBIGene'],fagene))
        candidate_set[core_complex[fagene]]=[]
        for hit in resp['results']['bindings']:
            candidate_set[core_complex[fagene]].append(hit['candidate']['value'])
            print('\t', hit['candidate']['value']) 

UBE2T 	 http://www.ncbi.nlm.nih.gov/gene/29089
	 http://www.ncbi.nlm.nih.gov/gene/3608
	 http://www.ncbi.nlm.nih.gov/gene/171392
	 http://www.ncbi.nlm.nih.gov/gene/340252
	 http://www.ncbi.nlm.nih.gov/gene/8125
	 http://www.ncbi.nlm.nih.gov/gene/54958


UBE2T is the only gene in the Fancomi Anemia core complex  
with optimal associations with other genes
Briefly those associated genes are:   

- ILF2 http://www.ncbi.nlm.nih.gov/gene/3608  
    a transcription factor required for T-cell expression of the interleukin 2 gene

- ZNF675 https://www.ncbi.nlm.nih.gov/gene/171392  
    the novel zinc finger protein TIZ may play a role during osteoclast differentiation by modulating TRAF6 signaling activity.

- ZNF680 http://www.ncbi.nlm.nih.gov/gene/340252  
    obsevered expressed in thyroid but fairly uncharaterizied

- ANP32A http://www.ncbi.nlm.nih.gov/gene/8125  
    expressed lymph nodes & bone marrow 
    The tumor suppressor acidic nuclear phosphoprotein 32 family, member A 

- TMEM160 http://www.ncbi.nlm.nih.gov/gene/54958  
    Not much to see here

----
Where to go next

- We can look for ideal matches in larger upstream regions but counter intiutivly larger regions average fewer associations because the number of distinct motifs between the larger regions grows faster than the number of motifs the regions will have in common.(and I dropped any that dipped below 50% similar)
- We can look for less ideal matches in the same 1k region 
- We can look for less ideal matches in larger regions